<a href="https://colab.research.google.com/github/smara97/FakeNews/blob/master/GP_mydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from google.colab import files,drive
import collections
import nltk
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from string import punctuation
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def calculate_languages_ratios(text):
   
    languages_ratios = {}
    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) 

    return languages_ratios

In [0]:
def detect_language(text):
    ratios = calculate_languages_ratios(text)
    most_rated_language = max(ratios, key=ratios.get)
    return most_rated_language

In [0]:
def tokenbyreg(text,pattern):
  tokenizer = RegexpTokenizer(pattern)
  return tokenizer.tokenize(text)

In [0]:
def removestop(text):
  stopwords_set = set(stopwords.words("english"))
  texs=[]
  for i in range(len(text)):
    word=text[i].lower()
    if word not in stopwords_set and word not in punctuation:
      texs.append(word)
  return texs

In [0]:
def stemtext(text):
  ps = PorterStemmer()
  stemmed_words=[]
  for w in text:
    stemmed_words.append(ps.stem(w))
  return stemmed_words

In [0]:
def lemtext(text):
  lem = WordNetLemmatizer()
  lemm_words=[]
  for w in text:
    lemm_words.append(lem.lemmatize(w))
  return lemm_words

In [10]:
alldata=pd.read_csv("/content/drive/My Drive/Fake news competitive/finalstraingv1.csv")
alldata=alldata.dropna()
alldata=alldata.drop([alldata.columns[0],alldata.columns[4]],axis=1)
alldata.head()

,title,text,author,label
0,WATCH: Anti-LGBT Attack At Equality House – SH...,WATCH: Anti-LGBT Attack At Equality House – SH...,Jordan Baranowski,1
1,"Politico Tries to Destroy Trump, But It Backfi...",You are here: Home / US / Politico Tries to De...,Regan Pifer,1
2,"an interview on ABC's ""This Week""",Independent voters have a ten-point margin in ...,bob-mcdonnell,0
3,Trump warning: the start of World War III and ...,"Chaosistan , Phenomenon of Terrorism by Gulam ...",Editorial,1
4,Obama’s Support of Erdogan Is a Stark Reminder...,WASHINGTON — It’s hard to remember today th...,Mark Landler,0


In [0]:
authors=alldata.author.values

In [0]:
indr,indn,inds,indt=[],[],[],[]
for i in range(alldata.shape[0]):
    try:
        st= alldata.loc[i,'author']
        op=""
        for w in st.split():
            if w!='noreply@blogger.com' and w!='(noreply@blogger.com)':
                op+=w+' '
            else:
                indn.append(i)
        if len(op.split(','))>3:
            #op="removed"
            indr.append(i)
        alldata.loc[i,'author']=op

        if detect_language(alldata.loc[i,'text'])!="english" :
          inds.append(i)
        elif  detect_language(alldata.loc[i,'title'])!="english":
          indt.append(i)

    except:
        continue

In [13]:
print(len(inds),len(indt))

696 7553


In [0]:
pattern = r'''(?x)
(?:[A-Z]\.)+(?:[A-Z])
| \w+(?:-\w+)*
| \$?\d+(?:\.\d+)?%?
| \.\.\.| [][.,;"'?():_`-]'''

In [53]:
stemtext(removestop(tokenbyreg("Last your chance to travle from away egypt to U.S.A ?",pattern)))

['last', 'chanc', 'travl', 'away', 'egypt', 'u.s.a']

In [67]:
lemtext(removestop(tokenbyreg("Last your chance to travle from away egypt to U.S.A ?",pattern)))

['last', 'chance', 'travle', 'away', 'egypt', 'u.s.a']